In [1]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [2]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [3]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN/'
valid_path = 'Brain tumor detection datasets/Br35H-Mask-RCNN/TEST/'

In [4]:
# Import the VGG16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [5]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [6]:
# useful for getting number of output classes
folders = glob('Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN/*')

In [7]:
folders

['Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN\\annotations_train.json',
 'Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN\\y0.jpg',
 'Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN\\y1.jpg',
 'Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN\\y10.jpg',
 'Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN\\y100.jpg',
 'Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN\\y101.jpg',
 'Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN\\y102.jpg',
 'Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN\\y103.jpg',
 'Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN\\y104.jpg',
 'Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN\\y105.jpg',
 'Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN\\y106.jpg',
 'Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN\\y107.jpg',
 'Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN\\y108.jpg',
 'Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN\\y109.jpg',
 'Brain tumor detection datasets/Br35H-Mask-RCNN/TRAI

In [8]:
# our layers - you can add more if you want
x = Flatten()(vgg16.output)

In [9]:
len(folders)

501

In [10]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [11]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [17]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [18]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('Brain tumor detection datasets/Br35H-Mask-RCNN/TRAIN',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 0 images belonging to 0 classes.


In [19]:
test_set = test_datagen.flow_from_directory('Brain tumor detection datasets/Br35H-Mask-RCNN/TEST',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 0 images belonging to 0 classes.


In [22]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-22-2d02736eff38>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


ValueError: Asked to retrieve element 0, but the Sequence has length 0